### m8.1stan

m8.1stan is the first model in the Statistical Rethinking book (pp. 249) using Stan.

Here we will use Turing's NUTS support, which is currently (2018) the originalNUTS by [Hoffman & Gelman]( http://www.stat.columbia.edu/~gelman/research/published/nuts.pdf) and not the one that's in Stan 2.18.2, i.e., Appendix A.5 in: https://arxiv.org/abs/1701.02434

The StatisticalRethinking pkg uses, e.g., Turing, CSV, DataFrames

In [1]:
using StatisticalRethinking

loaded


┌ Warning: Package Turing does not have CmdStan in its dependencies:
│ - If you have Turing checked out for development and have
│   added CmdStan as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with Turing
│ Loading CmdStan into Turing from project dependency, future warnings for Turing are suppressed.
└ @ nothing nothing:840


Read in rugged data as a DataFrame

In [2]:
d = CSV.read(joinpath(dirname(Base.pathof(StatisticalRethinking)), "..", "data",
    "rugged.csv"), delim=';');
# Show size of the DataFrame (should be 234x51)
size(d)

(234, 51)

Apply log() to each element in rgdppc_2000 column and add it as a new column

In [3]:
d = hcat(d, map(log, d[Symbol("rgdppc_2000")]));

Rename our col x1 => log_gdp

In [4]:
rename!(d, :x1 => :log_gdp);

Now we need to drop every row where rgdppc_2000 == missing

When this (https://github.com/JuliaData/DataFrames.jl/pull/1546) hits DataFrame it'll be conceptually easier: i.e., completecases!(d, :rgdppc_2000)

In [5]:
notisnan(e) = !ismissing(e)
dd = d[map(notisnan, d[:rgdppc_2000]), :];

Updated DataFrame dd size (should equal 170 x 52)

In [6]:
size(dd)

(170, 52)

Define the Turing model

In [7]:
@model m8_1stan(y, x₁, x₂) = begin
    σ ~ Truncated(Cauchy(0, 2), 0, Inf)
    βR ~ Normal(0, 10)
    βA ~ Normal(0, 10)
    βAR ~ Normal(0, 10)
    α ~ Normal(0, 100)

    for i ∈ 1:length(y)
        y[i] ~ Normal(α + βR * x₁[i] + βA * x₂[i] + βAR * x₁[i] * x₂[i], σ)
    end
end;

Test to see that the model is sane. Use 2000 for now, as in the book.
Need to set the same stepsize and adapt_delta as in Stan...

In [8]:
posterior = sample(m8_1stan(dd[:,:log_gdp], dd[:,:rugged], dd[:,:cont_africa]),
    Turing.NUTS(2000, 1000, 0.95));
# Describe the posterior samples
describe(posterior)

┌ Warning: Indexing with colon as row will create a copy in the future. Use `df[col_inds]` to get the columns without copying
│   caller = top-level scope at In[8]:1
└ @ Core In[8]:1
┌ Warning: Indexing with colon as row will create a copy in the future. Use `df[col_inds]` to get the columns without copying
│   caller = top-level scope at In[8]:1
└ @ Core In[8]:1
┌ Warning: Indexing with colon as row will create a copy in the future. Use `df[col_inds]` to get the columns without copying
│   caller = top-level scope at In[8]:1
└ @ Core In[8]:1
┌ Info: [Turing] looking for good initial eps...
└ @ Turing /Users/rob/.julia/packages/Turing/orJH9/src/samplers/support/hmc_core.jl:246
[NUTS{Union{}}] found initial ϵ: 0.05
└ @ Turing /Users/rob/.julia/packages/Turing/orJH9/src/samplers/support/hmc_core.jl:291
[NUTS] Sampling...  1%  ETA: 0:06:54
  ϵ:         0.021702349971125082
  α:         0.9890516933991319
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling...  2%  ETA: 0:05:49
  ϵ: 

4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 42%  ETA: 0:01:06
  ϵ:         0.030737127864859708
  α:         0.9821912777159816
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 43%  ETA: 0:01:05
  ϵ:         0.026438391658241963
  α:         0.9980480594684609
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 44%  ETA: 0:01:03
  ϵ:         0.02780498401755003
  α:         0.9379581388525683
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 45%  ETA: 0:01:02
  ϵ:         0.021497580222665725
  α:         0.997838993180469
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 46%  ETA: 0:01:01
  ϵ:         0.030820042201476643
  α:         0.9780183694318911
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 47%  ETA: 0:00:59
  ϵ:         0.02581329680132846
  α:         0.9602252978375723
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 48%  ETA: 0:00:58
  ϵ:         0.04447510506264669
  α:         0

[NUTS] Sampling... 86%  ETA: 0:00:15
  ϵ:         0.028862387932864265
  α:         0.9757542419609502
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 88%  ETA: 0:00:13
  ϵ:         0.028862387932864265
  α:         0.9098487923015703
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 89%  ETA: 0:00:12
  ϵ:         0.028862387932864265
  α:         0.9116645340587638
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 90%  ETA: 0:00:11
  ϵ:         0.028862387932864265
  α:         0.8741954363453688
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 90%  ETA: 0:00:10
  ϵ:         0.028862387932864265
  α:         0.98152618446337
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 92%  ETA: 0:00:09
  ϵ:         0.028862387932864265
  α:         1.0
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0]


[NUTS] Sampling... 93%  ETA: 0:00:08
  ϵ:         0.028862387932864265
  α:         0.997082288585899
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0

[NUTS] Finished with
  Running time        = 107.10928417000004;
  #lf / sample        = 0.0015;
  #evals / sample     = 44.931;
  pre-cond. metric    = [1.0, 1.0, 1.0, 1.0, 1.0].


[NUTS] Sampling...100% Time: 0:01:48


Iterations = 1:2000
Thinning interval = 1
Chains = 1
Samples per chain = 2000

Empirical Posterior Estimates:
              Mean           SD         Naive SE        MCSE         ESS   
       α    9.192198853  0.471485288 0.01054273155 0.03488051527  182.71345
  lf_num    0.001500000  0.067082039 0.00150000000 0.00150000000 2000.00000
      βA   -1.916864926  0.377451668 0.00844007588 0.02969740713  161.54207
      βR   -0.195354050  0.142021349 0.00317569391 0.01008324641  198.38394
       σ    0.968861418  0.295329191 0.00660376148 0.01871696223  248.96714
 elapsed    0.053554642  0.076105161 0.00170176314 0.00239230124 1012.03724
 epsilon    0.029681037  0.018393691 0.00041129543 0.00041492961 1965.11923
eval_num   44.931000000 25.521538227 0.57067894365 0.67314732101 1437.45232
     βAR    0.386290432  0.160630677 0.00359181114 0.00838013229  367.41360
      lp -249.858010884 17.850220615 0.39914306709 1.35731471511  172.95219
  lf_eps    0.029681037  0.018393691 0.00041129543 0.0

Example of a Turing run simulation output

In [11]:
m_08_1t_turing_result = "
        Mean           SD        Naive SE        MCSE         ESS
α    9.2140454953  0.416410339 0.00931121825 0.0303436655  188.324543
βA  -1.9414588557  0.373885658 0.00836033746 0.0583949856   40.994586
βR  -0.1987645549  0.158902372 0.00355316505 0.0128657961  152.541295
σ    0.9722532977  0.440031013 0.00983939257 0.0203736871  466.473854
βAR  0.3951414223  0.187780491 0.00419889943 0.0276680621   46.062071
";

Here's the map2stan output from rethinking

In [13]:
m_08_1_map2stan_result = "
       Mean StdDev lower 0.89 upper 0.89 n_eff Rhat
 a      9.24   0.14       9.03       9.47   291    1
 bR    -0.21   0.08      -0.32      -0.07   306    1
 bA    -1.97   0.23      -2.31      -1.58   351    1
 bAR    0.40   0.13       0.20       0.63   350    1
 sigma  0.95   0.05       0.86       1.03   566    1
";

#-

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*